# Cuaderno 3: Clases y funciones especiales

La interfaz Python de Gurobi define funciones y tipos de datos (clases) especialmente diseñados para la formulación de modelos de programación matemática. Revisaremos en este cuaderno los tipos `tuplelist` y `tupledict`, y la función `multidict`.

Para utilizarlos es necesario importar el módulo de Gurobi:

In [ ]:
import gurobipy as gp
from gurobipy import GRB

## La clase tuplelist

La clase `tuplelist` es un tipo especial de lista cuyos elementos son tuplas de longitud constante. Está diseñada para almacenar y manipular de manera eficiente los índices en los modelos.

In [ ]:
L = gp.tuplelist([(1, 2), (1, 3), (1, 4), (2, 5), (3, 4), (4, 3)])
print(L)

Un objeto de tipo `tuplelist` puede acceder a todas las funciones y métodos de una lista común:

In [ ]:
L.append((1,5))
print(L)
L.pop()
print(L)
L.insert(0, (1, 5))
print(L)

Adicionalmente, un objeto `tuplelist`tiene un método `select` que retorna una sublista con las tuplas que cumplan determinados criterios:

In [ ]:
print(L)
print(L.select('*','*'))
print()
print(L.select(1, '*'))  # primera componente igual a 1
print()
print(L.select([2,3], 4)) # primera componente igual a 2 ó 3, segunda componente igual a 4
print()
print(L.select('*', [3, 4])) # segunda componente igual a 3 ó 4

Notar que la selección de tuplas del ejemplo anterior también podría hacerse con inclusiones de listas (*list comprehensions*). Sin embargo, el método `select` es computacionalmente mucho más eficiente.

In [ ]:
# primera componente igual a 1
print([(i, j) for (i, j) in L if i==1])  
# primera componente igual a 2 ó 3, segunda componente igual a 4
print([(i, j) for (i, j) in L if i in [2,3] and j==4])  
# segunda componente igual a 3 ó 4
print([(i, j) for (i, j) in L if j in [3, 4]])  

## La clase tupledict

La clase `tupledict` representa un tipo especial de diccionarios cuyas claves son tuplas de longitud constante. Está diseñada para almacenar y manipular de manera eficiente los parámetros y variables en los modelos.

In [ ]:
# ejemplo capacidades de los arcos de un grafo
u = gp.tupledict({(1,2) : 3, 
               (1,3) : 4.5, 
               (1,4) : 5,
               (2,3) : 1.1,
               (4,3) : 0.33})
print(u)
print(type(u))
print(type(u.keys()))

Pueden usarse todas las fuciones de un diccionario común. La lista de claves es un objeto del tipo `tuplelist`.

In [ ]:
print(u[2,3])
print(u.keys())
u[1,4] = 6  # notar que los paréntesis no son necesarios para referirse a la clave
u[4,1] = 7  # es lo mismo que u[(4,1)] = 7
print(u)

Como la lista de claves es un `tuplelist` puede emplearse la función `select` para iterar sobre ella: 

In [ ]:
print('Nodos sucesores al 1 y sus capacidades:')
for i, j in u.keys().select(1, '*'):
    print('{}\t\t{}'.format(j, u[i,j]))


Adicionalmente, la clase `tupledict` tienen dos métodos diseñados especialmente para facilitar la creación de expresiones lineales: `sum` y `prod`.

El método `sum` suma aquellos valores del diccionario cuyas claves satisfagan un criterio de selección:

In [ ]:
print(u)
# sumar los valores indexados por tuplas donde la primera componente es 1
print(u.sum(1, '*'))  
# sumar los valores correspondientes a tuplas con primera componente 1, segunda componente 2 ó 3
print(u.sum(1, [2,3])) 
print(type(u.sum(1, [2,3])))

El método `prod` requiere de otro diccionario de coeficientes `c` que tenga las mismas claves que el diccionario actual `u`. Este método realiza las siguientes operaciones:
1. Para cada clave que satisface un criterio de selección, se multiplican los valores correspondientes de ambos diccionarios
2. Se suman los resultados de todos los productos


In [ ]:
# construimos un diccionario u indexado por tuplas
u = gp.tupledict({(1,2) : 3, 
               (1,3) : 4.5, 
               (1,4) : 5,
               (2,3) : 1.1,
               (4,3) : 4,
               (4,1) : 5})
# construimos un diccionario c indexado por las mismas tuplas
c = gp.tupledict({(1,2) : 2, 
               (1,3) : 1, 
               (1,4) : 2,
               (2,3) : 3,
               (4,3) : 4,
               (4,1) : 5})
print("u = {}".format(u))
print("c = {}".format(c))
# calcular sum_{i,j} c(i,j)*u(i,j)
print(u.prod(c, '*', '*'))
print(c.prod(u, '*', '*'))
# calcular c(1,2)*u(1,2) + c(1,3)*u(1,3) + c(1,4)*u(1,4)
print(u.prod(c, 1, '*'))
# calcular c(1,2)*u(1,2) + c(1,3)*u(1,3) 
print(u.prod(c, 1, [2,3]))

Observar que las funciones `sum` y `prod` retornan valores del tipo `LinExpr` (expresión lineal). Esto ocurre porque las funciones pueden ser aplicadas a diccionarios que contengan variables del modelo, para construir expresiones lineales a ser utilizadas en la función objetivo o en las restricciones:

In [ ]:
import random as rd

# definir listas con índices
I = [i+1 for i in range(5)]
J = [2*i for i in range(5)]
print(I)
print(J)
print('---')

# crear un objeto modelo
m = gp.Model('ejemplo')
# crear variables binarias x_i indexadas por I
x = m.addVars(I, vtype=GRB.BINARY, name="x")
m.update()
print(x)
print(type(x))
# construir la expresión lineal x2 + x3 + x4
print(x.sum([2,3,4]))
print('---')

# crear variables enteras y_{ij} indexadas por I x J
y = m.addVars(I, J, vtype=GRB.INTEGER, name='y')
m.update()
print(y)
print()
# construir un tupledict indexado por I x J con costos c_{ij}
c = gp.tupledict({(i, j) : rd.randint(1,10) for i in I for j in J})
print(c)
print()
# construir la expresión sum_{i in I} sum_{j in J} c_{ij} * y_{i,j}
print(y.prod(c, '*', '*'))
print()
# construir la expresión sum_{j in J} c_{1,j} * y_{1,j}
print(y.prod(c, 1, '*'))
print()
# construir la expresión sum_{i in I} sum_{j in {2,4}} c_{i,j} * y_{i,j}
print(y.prod(c, '*', [2,4]))


Al igual que lo que ocurre con la función `select`, las funciones `sum` y `prod` podrían implementarse usando lazos, pero su ventaja radica en que son computacionalmente más eficientes.

## La función multidict

Suponer que se tiene un diccionario en el que los *valores* son siempre *tuplas de longitud constante* $k$:

In [ ]:
D = {'a' : (1, 3, 5), 
     'b' : (2, 4, 6), 
     'c' : (1, -1, 0), 
     'd' : (4, 9, 8)} # cada valor es una tupla de longitud 3
print(D)

La función `multidict` se usa para separar este diccionario en una lista que contiene las claves de `D`, y $k$   diccionarios, cada uno formado por las claves y uno de los elementos de las tuplas:

In [ ]:
(claves, primera, segunda, tercera) = gp.multidict(D)
print(claves)
print(primera)
print(segunda)
print(tercera)

Las claves del diccionario original pueden ser a su vez tuplas. En este caso, la lista de claves es del tipo `tuplelist`. Esto es útil en la formulación de muchos modelos de optimización. 

In [ ]:
# Diccionario con datos de entrada:
# arcos : capacidades, costos
datos = {(1,2) : (3, 5), 
         (1,3) : (4, 4), 
         (1,4) : (5, 3),
         (2,3) : (1, 1),
         (4,3) : (0, 7)}
arcos, capacidades, costos = gp.multidict(datos)
print(arcos)
print(capacidades)
print(costos)

## Recordatorio: Expresiones generadoras

Las expresiones generadoras (o simplemente generadores) son parte de la sintaxis regular del lenguaje Python. Permiten construir estructuras iterables "sobre la marcha", generalmente para utilizarlas en funciones. 

Los generadores emplean la misma sintaxis que las inclusiones (*list comprehensions*).

Ejemplo: Dados

In [ ]:
u = gp.tupledict({(1,2) : 3, 
               (1,3) : 4.5, 
               (1,4) : 5,
               (2,3) : 1.1,
               (2,4) : 0.33})
d = gp.tupledict({1 : 1, 2: 2, 3 : 2, 4 : 3})
print(u)
print(d)


Suponer que queremos calcular el valor de $\sum_{j} d_j u_{2,j} = d_3 u_{2,3} + d_4 u_{2,4}$. 

Una posibilidad es usar inclusiones para construir una lista con los términos del sumatorio, y luego llamar a la función `quicksum` para sumar los elementos de la lista:

In [ ]:
L = [d[j]*u[i,j] for i,j in u.keys().select(2, '*')]
print(L)
print(gp.quicksum(L))

Otra posibilidad es usar directamente un generador como argumento de la función `quicksum`. En esta segunda alternativa, Python no construye explícitamente la lista, sino que va generando dinámicamente sus términos conforme los necesita para calcular la suma. Por lo tanto, esta alternativa requiere de menos memoria en el computador:

In [ ]:
print(gp.quicksum(d[j]*u[i,j] for i,j in u.keys().select(2, '*')))